In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset, Subset

from utils import *
from datasets import *
from mdav import *
from train import *
from models import *
from attacks import *

from sklearn.preprocessing import StandardScaler, LabelEncoder

import random
import time
import copy
from collections import Counter
import csv

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning

# Filter out ConvergenceWarning and FitFailedWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
# def seed_everything(seed=7):
#     np.random.seed(seed)
#     np.random.seed(seed)
#     random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(seed=7)

In [4]:
# Step 1: Get dataset

df=pd.read_csv('data/heart/cardio_train.csv', sep=';')
df.drop(columns=['id'], inplace=True)
df.dropna(inplace=True)

split_ratio = 0.8  # 80% for the first DataFrame, 20% for the second
# Perform the random split
mask = np.random.rand(len(df)) < split_ratio
trainset = df[mask]
testset = df[~mask]
# Reset the index of the new DataFrames if needed
trainset.reset_index(drop=True, inplace=True)
testset.reset_index(drop=True, inplace=True)


X_train = trainset.iloc[:,:-1].values
y_train = trainset.iloc[:,-1].values
X_test = testset.iloc[:,:-1].values
y_test = testset.iloc[:,-1].values

SC = StandardScaler()
X_train = SC.fit_transform(X_train)
X_test = SC.transform(X_test)

num_features = X_train.shape[-1]
num_classes = len(set(y_train))

# Randomly sample retain and forget sets
forget_ratio = 0.05
m = int(len(y_train)*forget_ratio)
idxs = np.arange(len(y_train))
random.shuffle(idxs)

retain_idxs = idxs[m:]
forget_idxs = idxs[:m]
X_retain = X_train[retain_idxs]
y_retain = y_train[retain_idxs]
X_forget = X_train[forget_idxs]
y_forget = y_train[forget_idxs]

print('X_train shape', X_train.shape)
print('X_train unique shape', np.unique(X_train, axis = 0).shape)
print('X_test shape', X_test.shape)
print('X_test unique shape', np.unique(X_test, axis = 0).shape)
print('X_retain shape', X_retain.shape)
print('X_retain unique shape', np.unique(X_retain, axis = 0).shape)
print('X_forget shape', X_forget.shape)
print('X_forget unique shape', np.unique(X_forget, axis = 0).shape)

# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.int64))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.int64))
retain_dataset = TensorDataset(torch.tensor(X_retain, dtype=torch.float32), torch.tensor(y_retain, dtype=torch.int64))
forget_dataset = TensorDataset(torch.tensor(X_forget, dtype=torch.float32), torch.tensor(y_forget, dtype=torch.int64))

# Create DataLoader instances
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
retain_loader = DataLoader(retain_dataset, batch_size=batch_size, shuffle=True)
forget_loader = DataLoader(forget_dataset, batch_size=batch_size, shuffle=False)


counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%s, Count=%d, Percentage=%.2f%%' % (k, v, per))
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
initial_model = MLPModel(num_features, 128, num_classes)
criterion = nn.CrossEntropyLoss()
lr = 1e-2
n_repeat = 3
max_epochs = 200
patience = None

X_train shape (56074, 11)
X_train unique shape (56048, 11)
X_test shape (13926, 11)
X_test unique shape (13925, 11)
X_retain shape (53271, 11)
X_retain unique shape (53248, 11)
X_forget shape (2803, 11)
X_forget unique shape (2803, 11)
Class=0, Count=28071, Percentage=50.06%
Class=1, Count=28003, Percentage=49.94%


In [5]:
# Step 2: Define and train M on D
train_accs = []
test_accs = []
mia_aucs = []
mia_advs = []
runtimes = []
for r in range(n_repeat):
    torch.cuda.empty_cache()
    model = copy.deepcopy(initial_model)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    t0 = time.time()
    model = train_model(model, train_loader, test_loader, criterion, optimizer, 
                        max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                        patience = patience)

    t1 = time.time()
    rt = t1-t0
    runtimes.append(rt)
    
    # Evaluate the model accuracy, and MIA
    model.eval()
    # Accuracy
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    train_accs.append(100.0*train_acc)
    test_accs.append(100.0*test_acc)
    #MIA
    idxs = np.arange(len(test_dataset))
    random.shuffle(idxs)
    rand_idxs = idxs[:m]
    logits_test, loss_test, test_labels = compute_attack_components(model, test_loader)
    logits_forget, loss_forget, forget_labels = compute_attack_components(model, forget_loader)
    attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                          forget_labels, test_labels[rand_idxs])
    auc = attack_result.get_result_with_max_auc().get_auc()
    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
    mia_aucs.append(100.0*auc)
    mia_advs.append(100.0*adv)

mean_runtime = np.mean(runtimes)
std_runtime = np.std(runtimes)
mean_train_acc = np.mean(train_accs)
std_train_acc = np.std(train_accs)
mean_test_acc = np.mean(test_accs)
std_test_acc = np.std(test_accs)
mean_mia_auc = np.mean(mia_aucs)
std_mia_auc = np.std(mia_aucs)
mean_mia_adv = np.mean(mia_advs)
std_mia_adv = np.std(mia_advs)

# Print the results
print('Training M on D time:{:0.2f}(±{:0.2f}) seconds'.format(mean_runtime, std_runtime))
print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_acc, std_train_acc))
print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_acc, std_test_acc))
print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_auc, std_mia_auc))
print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_adv, std_mia_adv))

# Save to CSV
csv_file_path = 'results/heart/mlp_m_d_fr={}.csv'.format(forget_ratio)

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
    writer.writerow(['Training Time', mean_runtime, std_runtime])
    writer.writerow(['Train Accuracy', mean_train_acc, std_train_acc])
    writer.writerow(['Test Accuracy', mean_test_acc, std_test_acc])
    writer.writerow(['MIA AUC', mean_mia_auc, std_mia_auc])
    writer.writerow(['MIA Advantage', mean_mia_adv, std_mia_adv])


Training M on D time:198.27(±2.01) seconds
Train accuracy:75.96(±0.14)%
Test accuracy:72.35(±0.04)%
MIA AUC:52.90(±0.27)%
MIA Advantage:4.51(±0.34)%


In [6]:
# Step 3: Train M_retain on D_retain
retain_accs = []
forget_accs = []
test_accs = []
mia_aucs = []
mia_advs = []
runtimes = []
for r in range(n_repeat):
    torch.cuda.empty_cache()
    model_ret = copy.deepcopy(initial_model)
    optimizer = optim.Adam(model_ret.parameters(), lr=lr)
    t0 = time.time()
    model_ret = train_model(model_ret, retain_loader, test_loader, criterion, optimizer, 
                    max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                        patience = patience)

    t1 = time.time()
    rt = t1-t0
    runtimes.append(rt)
    
    # Evaluate the model accuracy, and MIA
    model_ret.eval()
    # Accuracy
    retain_acc = accuracy(model_ret, retain_loader)
    test_acc = accuracy(model_ret, test_loader)
    forget_acc = accuracy(model_ret, forget_loader)
    retain_accs.append(100.0*retain_acc)
    forget_accs.append(100.0*forget_acc)
    test_accs.append(100.0*test_acc)
    #MIA
    logits_test, loss_test, test_labels = compute_attack_components(model_ret, test_loader)
    logits_forget, loss_forget, forget_labels = compute_attack_components(model_ret, forget_loader)
    attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                          forget_labels, test_labels[rand_idxs])
    auc = attack_result.get_result_with_max_auc().get_auc()
    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
    mia_aucs.append(100.0*auc)
    mia_advs.append(100.0*adv)
    

mean_retrain_runtime = np.mean(runtimes)
std_retrain_runtime = np.std(runtimes)
mean_retain_acc = np.mean(retain_accs)
std_retain_acc = np.std(retain_accs)
mean_forget_acc = np.mean(forget_accs)
std_forget_acc = np.std(forget_accs)
mean_retrain_test_acc = np.mean(test_accs)
std_retrain_test_acc = np.std(test_accs)
mean_retrain_mia_auc = np.mean(mia_aucs)
std_retrain_mia_auc = np.std(mia_aucs)
mean_retrain_mia_adv = np.mean(mia_advs)
std_retrain_mia_adv = np.std(mia_advs)

# Print the results
print('Retraining M on D_ret time:{:0.2f}(±{:0.2f}) seconds'.format(mean_retrain_runtime, std_retrain_runtime))
print('Retain accuracy:{:0.2f}(±{:0.2f})%'.format(mean_retain_acc, std_retain_acc))
print('Forget accuracy:{:0.2f}(±{:0.2f})%'.format(mean_forget_acc, std_forget_acc))
print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_retrain_test_acc, std_retrain_test_acc))
print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_retrain_mia_auc, std_retrain_mia_auc))
print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_retrain_mia_adv, std_retrain_mia_adv))

# Save to CSV
csv_retrain_file_path = 'results/heart/mlp_mret_dret_fr={}.csv'.format(forget_ratio)

with open(csv_retrain_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
    writer.writerow(['Retraining Time', mean_retrain_runtime, std_retrain_runtime])
    writer.writerow(['Retain Accuracy', mean_retain_acc, std_retain_acc])
    writer.writerow(['Forget Accuracy', mean_forget_acc, std_forget_acc])
    writer.writerow(['Test Accuracy', mean_retrain_test_acc, std_retrain_test_acc])
    writer.writerow(['MIA AUC', mean_retrain_mia_auc, std_retrain_mia_auc])
    writer.writerow(['MIA Advantage', mean_retrain_mia_adv, std_retrain_mia_adv])


Retraining M on D_ret time:186.73(±0.46) seconds
Retain accuracy:75.54(±0.03)%
Forget accuracy:72.14(±0.20)%
Test accuracy:72.77(±0.20)%
MIA AUC:50.12(±0.18)%
MIA Advantage:1.99(±0.34)%


# k-anonymity

In [7]:
# Step 1: k-anonymize and prepare D_k
ft_epochs_list = [5]
for ft_epochs in ft_epochs_list:
    K = [3, 5, 10, 20, 80, 160, 320, 640]
    for k in K:
        runtimes_k = []
        t0 = time.time()
        centroids, clusters, labels, X_train_k, y_train_k = mdav(copy.deepcopy(X_train), copy.deepcopy(y_train), k)
        print_stats(clusters, centroids)
        print('Shape of X_train_k:{}, y_train_k:{}'.format(X_train_k.shape, y_train_k.shape))
         # Create TensorDatasets
        train_dataset_k = TensorDataset(torch.tensor(X_train_k, dtype=torch.float32), torch.tensor(y_train_k, dtype=torch.int64))
        train_loader_k = DataLoader(train_dataset_k, batch_size=batch_size, shuffle=True)
        t1 = time.time()
        rt_k = t1- t0
        runtimes_k.append(rt_k)

        train_accs_k = []
        test_accs_k = []
        mia_aucs_k = []
        mia_advs_k = []
        runtimes_train_k = []

        train_accs_k_D = []
        test_accs_k_D = []
        mia_aucs_k_D = []
        mia_advs_k_D = []
        runtimes_train_k_D = []

        retain_accs_k_ret = []
        forget_accs_k_ret = []
        test_accs_k_ret = []
        mia_aucs_k_ret = []
        mia_advs_k_ret = []
        runtimes_train_k_ret = []

        for r in range(n_repeat):
            torch.cuda.empty_cache()
            model_k = copy.deepcopy(initial_model)
            optimizer = optim.Adam(model_k.parameters(), lr=lr)
            t0 = time.time()
            model_k = train_model(model_k, train_loader_k, test_loader, criterion, optimizer, 
                            max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                            patience = patience)

            t1 = time.time()
            rt_train = t1- t0
            runtimes_train_k.append(rt_train)

            # Evaluate the model accuracy, and MIA
            model_k.eval()
            #Accuracy
            train_acc = accuracy(model_k, train_loader)
            test_acc = accuracy(model_k, test_loader)
            train_accs_k.append(100.0*train_acc)
            test_accs_k.append(100.0*test_acc)
            #MIA
            logits_test, loss_test, test_labels = compute_attack_components(model_k, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k.append(100.0*auc)
            mia_advs_k.append(100.0*adv)

            model_k_D = copy.deepcopy(model_k)
            torch.cuda.empty_cache()
            optimizer = optim.Adam(model_k_D.parameters(), lr=lr)
            t0 = time.time()
            model_k_D = train_model(model_k_D, train_loader, test_loader, criterion, optimizer, 
                                ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                                  patience = patience)

            t1 = time.time()
            rt = t1-t0
            runtimes_train_k_D.append(rt)

            # Evaluate the model accuracy, and MIA
            model_k_D.eval()
            #Accuracy
            train_acc = accuracy(model_k_D, train_loader)
            test_acc = accuracy(model_k_D, test_loader)
            train_accs_k_D.append(100.0*train_acc)
            test_accs_k_D.append(100.0*test_acc)
            #MIA
            logits_test, loss_test, test_labels = compute_attack_components(model_k_D, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_D, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k_D.append(100.0*auc)
            mia_advs_k_D.append(100.0*adv)

            model_k_ret = copy.deepcopy(model_k)
            torch.cuda.empty_cache()
            optimizer = optim.Adam(model_k_ret.parameters(), lr=lr)
            t0 = time.time()
            model_k_ret = train_model(model_k_ret, retain_loader, test_loader, criterion, optimizer, 
                                ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                                  patience = patience)

            t1 = time.time()
            rt = t1-t0
            runtimes_train_k_ret.append(rt)
            # Evaluate the model accuracy, and MIA
            model_k_ret.eval()
            #Accuracy
            retain_acc = accuracy(model_k_ret, retain_loader)
            forget_acc = accuracy(model_k_ret, forget_loader)
            test_acc = accuracy(model_k_ret, test_loader)
            retain_accs_k_ret.append(100.0*retain_acc)
            forget_accs_k_ret.append(100.0*forget_acc)
            test_accs_k_ret.append(100.0*test_acc)
            #MIA
            logits_test, loss_test, test_labels = compute_attack_components(model_k_ret, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_ret, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k_ret.append(100.0*auc)
            mia_advs_k_ret.append(100.0*adv)


        # Anonymizing D and training M_k on D_k
        mean_anonymize_time = np.mean(runtimes_k)
        std_anonymize_time = np.std(runtimes_k)
        mean_train_k_time = np.mean(runtimes_train_k)
        std_train_k_time = np.std(runtimes_train_k)
        mean_train_k_acc = np.mean(train_accs_k)
        std_train_k_acc = np.std(train_accs_k)
        mean_test_k_acc = np.mean(test_accs_k)
        std_test_k_acc = np.std(test_accs_k)
        mean_mia_k_auc = np.mean(mia_aucs_k)
        std_mia_k_auc = np.std(mia_aucs_k)
        mean_mia_k_adv = np.mean(mia_advs_k)
        std_mia_k_adv = np.std(mia_advs_k)

        # Finetuning M_k on D
        mean_finetune_D_time = np.mean(runtimes_train_k_D)
        std_finetune_D_time = np.std(runtimes_train_k_D)
        mean_finetune_D_train_acc = np.mean(train_accs_k_D)
        std_finetune_D_train_acc = np.std(train_accs_k_D)
        mean_finetune_D_test_acc = np.mean(test_accs_k_D)
        std_finetune_D_test_acc = np.std(test_accs_k_D)
        mean_finetune_D_mia_auc = np.mean(mia_aucs_k_D)
        std_finetune_D_mia_auc = np.std(mia_aucs_k_D)
        mean_finetune_D_mia_adv = np.mean(mia_advs_k_D)
        std_finetune_D_mia_adv = np.std(mia_advs_k_D)

        # Finetuning M_k on D_ret
        mean_finetune_D_ret_time = np.mean(runtimes_train_k_ret)
        std_finetune_D_ret_time = np.std(runtimes_train_k_ret)
        mean_finetune_D_ret_train_acc = np.mean(retain_accs_k_ret)
        std_finetune_D_ret_train_acc = np.std(retain_accs_k_ret)
        mean_finetune_D_ret_forget_acc = np.mean(forget_accs_k_ret)
        std_finetune_D_ret_forget_acc = np.std(forget_accs_k_ret)
        mean_finetune_D_ret_test_acc = np.mean(test_accs_k_ret)
        std_finetune_D_ret_test_acc = np.std(test_accs_k_ret)
        mean_finetune_D_ret_mia_auc = np.mean(mia_aucs_k_ret)
        std_finetune_D_ret_mia_auc = np.std(mia_aucs_k_ret)
        mean_finetune_D_ret_mia_adv = np.mean(mia_advs_k_ret)
        std_finetune_D_ret_mia_adv = np.std(mia_advs_k_ret)



        # Print the results
        print('----------------------------------------')
        print('k=', k, 'Fine-tuning epochs=', ft_epochs)
        print('----------------------------------------')
        print('-----Anonymizing D and training M_k on D_k-----')
        print('Anonymizing D time:{:0.2f}(±{:0.2f})'.format(mean_anonymize_time, std_anonymize_time))
        print('Training M_k on D_k time:{:0.2f}(±{:0.2f})'.format(mean_train_k_time, std_train_k_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_k_acc, std_train_k_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_k_acc, std_test_k_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_auc, std_mia_k_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_adv, std_mia_k_adv))

        print('-----Finetuning M_k on D-----')
        print('Training M_k on D time:{:0.2f}(±{:0.2f})'.format(mean_finetune_D_time, std_finetune_D_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_train_acc, std_finetune_D_train_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_test_acc, std_finetune_D_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_auc, std_finetune_D_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_adv, std_finetune_D_mia_adv))

        print('-----Finetuning M_k on D_ret-----')
        print('Finetuning M_k on D_retain time:{:0.2f}(±{:0.2f}) seconds'.format(mean_finetune_D_ret_time, std_finetune_D_ret_time))
        print('Retain accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc))
        print('Forget accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv))
        print('----------------------------------------')

        # Save to CSV
        csv_anonymize_file_path = 'results/heart/mlp_mk={}_dk_fr={}.csv'.format(k, forget_ratio)
        csv_finetune_D_file_path = 'results/heart/mlp_mk={}_d_fr={}_epochs={}.csv'.format(k, forget_ratio, ft_epochs)
        csv_finetune_D_ret_file_path = 'results/heart/mlp_mk={}_dret_fr={}_epochs={}.csv'.format(k, forget_ratio, ft_epochs)

        # Writing to CSV for anonymizing, finetuning on D, and finetuning on D_ret
        with open(csv_anonymize_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Anonymizing Time', mean_anonymize_time, std_anonymize_time])
            writer.writerow(['Training M_k on D_k Time', mean_train_k_time, std_train_k_time])
            writer.writerow(['Train Accuracy', mean_train_k_acc, std_train_k_acc])
            writer.writerow(['Test Accuracy', mean_test_k_acc, std_test_k_acc])
            writer.writerow(['MIA AUC', mean_mia_k_auc, std_mia_k_auc])
            writer.writerow(['MIA Advantage', mean_mia_k_adv, std_mia_k_adv])

        with open(csv_finetune_D_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Training M_k on D Time', mean_finetune_D_time, std_finetune_D_time])
            writer.writerow(['Train Accuracy', mean_finetune_D_train_acc, std_finetune_D_train_acc])
            writer.writerow(['Test Accuracy', mean_finetune_D_test_acc, std_finetune_D_test_acc])
            writer.writerow(['MIA AUC', mean_finetune_D_mia_auc, std_finetune_D_mia_auc])
            writer.writerow(['MIA Advantage', mean_finetune_D_mia_adv, std_finetune_D_mia_adv])

        with open(csv_finetune_D_ret_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Finetuning M_k on D_retain Time', mean_finetune_D_ret_time, std_finetune_D_ret_time])
            writer.writerow(['Retain Accuracy', mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc])
            writer.writerow(['Forget Accuracy', mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc])
            writer.writerow(['Test Accuracy', mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc])
            writer.writerow(['MIA AUC', mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc])
            writer.writerow(['MIA Advantage', mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv])

 98%|████████████████████████████████████████████████████████████████████████▍ | 54903/56074 [01:29<00:00, 7684.86it/s]D:\machine unlearning\josep idea\mdav.py:103: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_k = np.vstack(np.repeat(c.mean(0).reshape(1, -1), len(c), axis = 0) for c in clusters)
100%|███████████████████████████████████████████████████████████████████████████| 56074/56074 [01:30<00:00, 621.11it/s]


Number of clusters: 18691
Mean of mean distances to centroids: 3.9142376999754136
Shape of X_train_k:(56074, 11), y_train_k:(56074,)
----------------------------------------
k= 3 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_k on D_k-----
Anonymizing D time:90.69(±0.00)
Training M_k on D_k time:195.12(±1.16)
Train accuracy:73.81(±0.03)%
Test accuracy:72.65(±0.07)%
MIA AUC:51.42(±0.14)%
MIA Advantage:3.31(±0.13)%
-----Finetuning M_k on D-----
Training M_k on D time:5.08(±0.06)
Train accuracy:73.74(±0.08)%
Test accuracy:73.77(±0.08)%
MIA AUC:50.46(±0.41)%
MIA Advantage:2.63(±0.77)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.78(±0.04) seconds
Retain accuracy:73.73(±0.08)%
Forget accuracy:72.45(±0.24)%
Test accuracy:73.87(±0.03)%
MIA AUC:51.03(±0.68)%
MIA Advantage:2.95(±0.64)%
----------------------------------------


 99%|████████████████████████████████████████████████████████████████████████▌| 55785/56074 [00:56<00:00, 12483.04it/s]D:\machine unlearning\josep idea\mdav.py:103: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_k = np.vstack(np.repeat(c.mean(0).reshape(1, -1), len(c), axis = 0) for c in clusters)
100%|███████████████████████████████████████████████████████████████████████████| 56074/56074 [00:57<00:00, 976.16it/s]


Number of clusters: 11214
Mean of mean distances to centroids: 3.905743138894533
Shape of X_train_k:(56074, 11), y_train_k:(56074,)
----------------------------------------
k= 5 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_k on D_k-----
Anonymizing D time:57.70(±0.00)
Training M_k on D_k time:193.46(±0.21)
Train accuracy:73.21(±0.12)%
Test accuracy:72.41(±0.18)%
MIA AUC:51.65(±0.73)%
MIA Advantage:3.82(±1.06)%
-----Finetuning M_k on D-----
Training M_k on D time:5.06(±0.03)
Train accuracy:73.67(±0.03)%
Test accuracy:73.64(±0.12)%
MIA AUC:50.43(±0.22)%
MIA Advantage:3.90(±0.70)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.67(±0.03) seconds
Retain accuracy:73.82(±0.08)%
Forget accuracy:72.42(±0.28)%
Test accuracy:73.71(±0.06)%
MIA AUC:51.04(±0.22)%
MIA Advantage:2.31(±0.37)%
----------------------------------------


 98%|███████████████████████████████████████████████████████████████████████▌ | 54940/56074 [00:28<00:00, 16529.46it/s]D:\machine unlearning\josep idea\mdav.py:103: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_k = np.vstack(np.repeat(c.mean(0).reshape(1, -1), len(c), axis = 0) for c in clusters)
100%|██████████████████████████████████████████████████████████████████████████| 56074/56074 [00:28<00:00, 1949.55it/s]


...

Number of clusters: 350
Mean of mean distances to centroids: 3.760483100624267
Shape of X_train_k:(56074, 11), y_train_k:(56074,)
----------------------------------------
k= 160 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_k on D_k-----
Anonymizing D time:1.88(±0.00)
Training M_k on D_k time:201.51(±3.82)
Train accuracy:70.96(±0.10)%
Test accuracy:71.54(±0.07)%
MIA AUC:50.45(±0.15)%
MIA Advantage:2.72(±0.24)%
-----Finetuning M_k on D-----
Training M_k on D time:5.45(±0.17)
Train accuracy:73.51(±0.17)%
Test accuracy:73.84(±0.10)%
MIA AUC:50.84(±0.23)%
MIA Advantage:2.93(±0.65)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.82(±0.16) seconds
Retain accuracy:73.67(±0.06)%
Forget accuracy:72.66(±0.26)%
Test accuracy:73.94(±0.04)%
MIA AUC:50.55(±0.57)%
MIA Advantage:2.76(±0.25)%
----------------------------------------


 87%|███████████████████████████████████████████████████████████████▎         | 48640/56074 [00:00<00:00, 75257.19it/s]D:\machine unlearning\josep idea\mdav.py:103: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_k = np.vstack(np.repeat(c.mean(0).reshape(1, -1), len(c), axis = 0) for c in clusters)
100%|█████████████████████████████████████████████████████████████████████████| 56074/56074 [00:00<00:00, 58874.24it/s]


Number of clusters: 175
Mean of mean distances to centroids: 3.713880157518731
Shape of X_train_k:(56074, 11), y_train_k:(56074,)
----------------------------------------
k= 320 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_k on D_k-----
Anonymizing D time:0.97(±0.00)
Training M_k on D_k time:196.22(±1.06)
Train accuracy:70.45(±0.39)%
Test accuracy:70.78(±0.50)%
MIA AUC:51.07(±0.51)%
MIA Advantage:2.95(±0.44)%
-----Finetuning M_k on D-----
Training M_k on D time:5.11(±0.03)
Train accuracy:73.51(±0.07)%
Test accuracy:73.97(±0.01)%
MIA AUC:50.08(±0.12)%
MIA Advantage:3.25(±0.53)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.86(±0.13) seconds
Retain accuracy:73.60(±0.01)%
Forget accuracy:72.35(±0.20)%
Test accuracy:73.94(±0.08)%
MIA AUC:50.73(±0.71)%
MIA Advantage:3.46(±0.32)%
----------------------------------------


 62%|████████████████████████████████████████████▉                            | 34560/56074 [00:00<00:00, 94145.31it/s]D:\machine unlearning\josep idea\mdav.py:103: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data_k = np.vstack(np.repeat(c.mean(0).reshape(1, -1), len(c), axis = 0) for c in clusters)
100%|████████████████████████████████████████████████████████████████████████| 56074/56074 [00:00<00:00, 114081.31it/s]


Number of clusters: 87
Mean of mean distances to centroids: 3.638771284768852
Shape of X_train_k:(56074, 11), y_train_k:(56074,)
----------------------------------------
k= 640 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_k on D_k-----
Anonymizing D time:0.50(±0.00)
Training M_k on D_k time:195.96(±2.21)
Train accuracy:68.59(±0.23)%
Test accuracy:69.09(±0.28)%
MIA AUC:51.38(±0.96)%
MIA Advantage:3.78(±1.08)%
-----Finetuning M_k on D-----
Training M_k on D time:5.10(±0.04)
Train accuracy:73.44(±0.01)%
Test accuracy:74.02(±0.10)%
MIA AUC:50.30(±0.25)%
MIA Advantage:3.25(±0.35)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.74(±0.04) seconds
Retain accuracy:73.51(±0.07)%
Forget accuracy:72.29(±0.19)%
Test accuracy:73.92(±0.05)%
MIA AUC:50.41(±0.27)%
MIA Advantage:2.51(±0.44)%
----------------------------------------


# Differential privacy

In [8]:
# Step 1: k-anonymize and prepare D_k
ft_epochs_list = [5]
for ft_epochs in ft_epochs_list:
    EPS = [0.5, 2.5, 5., 25., 50., 100., 250, 500, 1000]
    for eps in EPS:
        dp_train_data = pd.read_csv('dp_data/heart/dp_heart_eps={}.csv'.format(eps), sep=',')
        # Drop useless columns
        dp_train_data.dropna(inplace=True)
        # convert the income column to 0 or 1 and then drop the column for the feature vectors
        # creating the feature vector 
        X_train_dp = dp_train_data.drop('cardio', axis =1)
        # target values
        y_train_dp = dp_train_data['cardio'].values
        # pass the data through the full_pipeline
        X_train_dp = SC.fit_transform(X_train_dp)
        # Create TensorDatasets
        train_dataset_k = TensorDataset(torch.tensor(X_train_dp, dtype=torch.float32), torch.tensor(y_train_dp, dtype=torch.int64))
        train_loader_k = DataLoader(train_dataset_k, batch_size=batch_size, shuffle=True)

        train_accs_k = []
        test_accs_k = []
        mia_aucs_k = []
        mia_advs_k = []
        runtimes_train_k = []

        train_accs_k_D = []
        test_accs_k_D = []
        mia_aucs_k_D = []
        mia_advs_k_D = []
        runtimes_train_k_D = []

        retain_accs_k_ret = []
        forget_accs_k_ret = []
        test_accs_k_ret = []
        mia_aucs_k_ret = []
        mia_advs_k_ret = []
        runtimes_train_k_ret = []

        for r in range(n_repeat):
            torch.cuda.empty_cache()
            model_k = copy.deepcopy(initial_model)
            optimizer = optim.Adam(model_k.parameters(), lr=lr)
            t0 = time.time()
            model_k = train_model(model_k, train_loader_k, test_loader, criterion, optimizer, 
                            max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                            patience = patience)

            t1 = time.time()
            rt_train = t1- t0
            runtimes_train_k.append(rt_train)

            # Evaluate the model accuracy, and MIA
            model_k.eval()
            #Accuracy
            train_acc = accuracy(model_k, train_loader)
            test_acc = accuracy(model_k, test_loader)
            train_accs_k.append(100.0*train_acc)
            test_accs_k.append(100.0*test_acc)
            #MIA
            idxs = np.arange(len(test_dataset))
            random.shuffle(idxs)
            rand_idxs = idxs[:m]
            logits_test, loss_test, test_labels = compute_attack_components(model_k, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k.append(100.0*auc)
            mia_advs_k.append(100.0*adv)

            model_k_D = copy.deepcopy(model_k)
            torch.cuda.empty_cache()
            optimizer = optim.Adam(model_k_D.parameters(), lr=lr)
            t0 = time.time()
            model_k_D = train_model(model_k_D, train_loader, test_loader, criterion, optimizer, 
                                ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                                  patience = patience)

            t1 = time.time()
            rt = t1-t0
            runtimes_train_k_D.append(rt)

            # Evaluate the model accuracy, and MIA
            model_k_D.eval()
            #Accuracy
            train_acc = accuracy(model_k_D, train_loader)
            test_acc = accuracy(model_k_D, test_loader)
            train_accs_k_D.append(100.0*train_acc)
            test_accs_k_D.append(100.0*test_acc)
            #MIA
            logits_test, loss_test, test_labels = compute_attack_components(model_k_D, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_D, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k_D.append(100.0*auc)
            mia_advs_k_D.append(100.0*adv)

            model_k_ret = copy.deepcopy(model_k)
            torch.cuda.empty_cache()
            optimizer = optim.Adam(model_k_ret.parameters(), lr=lr)
            t0 = time.time()
            model_k_ret = train_model(model_k_ret, retain_loader, test_loader, criterion, optimizer, 
                                ft_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                                  patience = patience)

            t1 = time.time()
            rt = t1-t0
            runtimes_train_k_ret.append(rt)
            # Evaluate the model accuracy, and MIA
            model_k_ret.eval()
            #Accuracy
            retain_acc = accuracy(model_k_ret, retain_loader)
            forget_acc = accuracy(model_k_ret, forget_loader)
            test_acc = accuracy(model_k_ret, test_loader)
            retain_accs_k_ret.append(100.0*retain_acc)
            forget_accs_k_ret.append(100.0*forget_acc)
            test_accs_k_ret.append(100.0*test_acc)
            #MIA
            logits_test, loss_test, test_labels = compute_attack_components(model_k_ret, test_loader)
            logits_forget, loss_forget, forget_labels = compute_attack_components(model_k_ret, forget_loader)
            attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                                  forget_labels, test_labels[rand_idxs])
            auc = attack_result.get_result_with_max_auc().get_auc()
            adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
            mia_aucs_k_ret.append(100.0*auc)
            mia_advs_k_ret.append(100.0*adv)


        # Anonymizing D and training M_k on D_k
        mean_train_k_time = np.mean(runtimes_train_k)
        std_train_k_time = np.std(runtimes_train_k)
        mean_train_k_acc = np.mean(train_accs_k)
        std_train_k_acc = np.std(train_accs_k)
        mean_test_k_acc = np.mean(test_accs_k)
        std_test_k_acc = np.std(test_accs_k)
        mean_mia_k_auc = np.mean(mia_aucs_k)
        std_mia_k_auc = np.std(mia_aucs_k)
        mean_mia_k_adv = np.mean(mia_advs_k)
        std_mia_k_adv = np.std(mia_advs_k)

        # Finetuning M_k on D
        mean_finetune_D_time = np.mean(runtimes_train_k_D)
        std_finetune_D_time = np.std(runtimes_train_k_D)
        mean_finetune_D_train_acc = np.mean(train_accs_k_D)
        std_finetune_D_train_acc = np.std(train_accs_k_D)
        mean_finetune_D_test_acc = np.mean(test_accs_k_D)
        std_finetune_D_test_acc = np.std(test_accs_k_D)
        mean_finetune_D_mia_auc = np.mean(mia_aucs_k_D)
        std_finetune_D_mia_auc = np.std(mia_aucs_k_D)
        mean_finetune_D_mia_adv = np.mean(mia_advs_k_D)
        std_finetune_D_mia_adv = np.std(mia_advs_k_D)

        # Finetuning M_k on D_ret
        mean_finetune_D_ret_time = np.mean(runtimes_train_k_ret)
        std_finetune_D_ret_time = np.std(runtimes_train_k_ret)
        mean_finetune_D_ret_train_acc = np.mean(retain_accs_k_ret)
        std_finetune_D_ret_train_acc = np.std(retain_accs_k_ret)
        mean_finetune_D_ret_forget_acc = np.mean(forget_accs_k_ret)
        std_finetune_D_ret_forget_acc = np.std(forget_accs_k_ret)
        mean_finetune_D_ret_test_acc = np.mean(test_accs_k_ret)
        std_finetune_D_ret_test_acc = np.std(test_accs_k_ret)
        mean_finetune_D_ret_mia_auc = np.mean(mia_aucs_k_ret)
        std_finetune_D_ret_mia_auc = np.std(mia_aucs_k_ret)
        mean_finetune_D_ret_mia_adv = np.mean(mia_advs_k_ret)
        std_finetune_D_ret_mia_adv = np.std(mia_advs_k_ret)

        # Print the results
        print('----------------------------------------')
        print('Epsilon=', eps, 'Fine-tuning epochs=', ft_epochs)
        print('----------------------------------------')
        print('-----Anonymizing D and training M_dp on D_dp-----')
        print('Training M_k on D_k time:{:0.2f}(±{:0.2f})'.format(mean_train_k_time, std_train_k_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_k_acc, std_train_k_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_k_acc, std_test_k_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_auc, std_mia_k_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_adv, std_mia_k_adv))

        print('-----Finetuning M_k on D-----')
        print('Training M_k on D time:{:0.2f}(±{:0.2f})'.format(mean_finetune_D_time, std_finetune_D_time))
        print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_train_acc, std_finetune_D_train_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_test_acc, std_finetune_D_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_auc, std_finetune_D_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_adv, std_finetune_D_mia_adv))

        print('-----Finetuning M_k on D_ret-----')
        print('Finetuning M_k on D_retain time:{:0.2f}(±{:0.2f}) seconds'.format(mean_finetune_D_ret_time, std_finetune_D_ret_time))
        print('Retain accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc))
        print('Forget accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc))
        print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc))
        print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc))
        print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv))
        print('----------------------------------------')

        # Save to CSV
        csv_anonymize_file_path = 'results/heart/mlp_mdp_eps={}_fr={}.csv'.format(eps, forget_ratio)
        csv_finetune_D_file_path = 'results/heart/mlp_mdpd_eps={}_fr={}_epochs={}.csv'.format(eps, forget_ratio, ft_epochs)
        csv_finetune_D_ret_file_path = 'results/heart/mlp_mdpret_eps={}_fr={}_epochs={}.csv'.format(eps, forget_ratio, ft_epochs)

        # Writing to CSV for anonymizing, finetuning on D, and finetuning on D_ret
        with open(csv_anonymize_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Training M_k on D_k Time', mean_train_k_time, std_train_k_time])
            writer.writerow(['Train Accuracy', mean_train_k_acc, std_train_k_acc])
            writer.writerow(['Test Accuracy', mean_test_k_acc, std_test_k_acc])
            writer.writerow(['MIA AUC', mean_mia_k_auc, std_mia_k_auc])
            writer.writerow(['MIA Advantage', mean_mia_k_adv, std_mia_k_adv])

        with open(csv_finetune_D_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Training M_k on D Time', mean_finetune_D_time, std_finetune_D_time])
            writer.writerow(['Train Accuracy', mean_finetune_D_train_acc, std_finetune_D_train_acc])
            writer.writerow(['Test Accuracy', mean_finetune_D_test_acc, std_finetune_D_test_acc])
            writer.writerow(['MIA AUC', mean_finetune_D_mia_auc, std_finetune_D_mia_auc])
            writer.writerow(['MIA Advantage', mean_finetune_D_mia_adv, std_finetune_D_mia_adv])

        with open(csv_finetune_D_ret_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
            writer.writerow(['Finetuning M_k on D_retain Time', mean_finetune_D_ret_time, std_finetune_D_ret_time])
            writer.writerow(['Retain Accuracy', mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc])
            writer.writerow(['Forget Accuracy', mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc])
            writer.writerow(['Test Accuracy', mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc])
            writer.writerow(['MIA AUC', mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc])
            writer.writerow(['MIA Advantage', mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv])

----------------------------------------
Epsilon= 0.5 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_dp on D_dp-----
Training M_k on D_k time:254.24(±8.19)
Train accuracy:50.36(±0.76)%
Test accuracy:50.59(±0.54)%
MIA AUC:50.79(±0.76)%
MIA Advantage:2.60(±0.34)%
-----Finetuning M_k on D-----
Training M_k on D time:5.08(±0.11)
Train accuracy:73.20(±0.05)%
Test accuracy:73.52(±0.03)%
MIA AUC:50.86(±0.73)%
MIA Advantage:2.71(±0.43)%
-----Finetuning M_k on D_ret-----
Finetuning M_k on D_retain time:4.86(±0.08) seconds
Retain accuracy:73.42(±0.22)%
Forget accuracy:72.20(±0.19)%
Test accuracy:73.73(±0.43)%
MIA AUC:50.36(±0.61)%
MIA Advantage:3.03(±0.64)%
----------------------------------------
----------------------------------------
Epsilon= 2.5 Fine-tuning epochs= 5
----------------------------------------
-----Anonymizing D and training M_dp on D_dp-----
Training M_k on D_k time:253.10(±0.13)
Train accuracy:51.51(±0.87)%
Test accuracy:51.7